# Step 1. Data Preparation for Modeling
----

### Objective 
1. Glue ETL로 수행 이후 S3에 파티션되어 저장된 데이터들을 통합 후에 학습셋/검증셋으로 분리하여 S3에 저장합니다.

In [ ]:
!pip install joblib
!pip install xgboost
!pip install lightgbm

In [2]:
import boto3
import os
import joblib
import numpy as np
import pandas as pd

s3 = boto3.client('s3')
feature_dir_name = 'modeling-feature-190808'     
bucket = 'analytics-preprocessed-daekeun'    # s3 bucket name
prefix_source = 'homecredit/train-csv'    # source directory
prefix_target = 'homecredit/{}'.format(feature_dir_name)

- Region과 IAM Role을 확인합니다.

In [3]:
from sagemaker import get_execution_role
region = boto3.Session().region_name
role = get_execution_role() # get_execution_role() function is to search IAM role
print((region, role))

('us-west-2', 'arn:aws:iam::143656149352:role/service-role/AmazonSageMaker-ExecutionRole-20190803T155872')


<br>

## 1. Data Loading and Transformation
----
- S3에 파티션되어 저장된 데이터들을 하나의 Pandas DataFrame으로 통합합니다. 데모에선 편의상 `pd.concat()`으로 여러 리스트들을 불러온 후 oncatenation을 수행하였지만, 이 방법은 매번 리스트를 새로 생성하므로 불필요한 메모리 낭비가 발생한다는 점을 주의하세요.
- Binary Classification은 Supervisied Learning이므로 feature와 label를 분리합니다 (X and y). **`sk_id_curr` 컬럼은 고객의 고유 id로 학습 시에 불필요하므로 제거해야 합니다.**
- 결측값들을 모두 0으로 대체합니다. 본 데모에선 편의상 0으로 간단하게 대체하였지만, 데이터 특성과 각 컬럼의 결측치 분포에 따라 적절한 결측치 채움 기법을(imputation) 적용하는 것을 권장합니다. 
    - 참조: https://scikit-learn.org/stable/modules/impute.html

In [4]:
%%time
objs = []
response = s3.list_objects(Bucket = bucket, Prefix = prefix_source)

for obj in response['Contents']:
    objs.append(obj['Key'])

train_df = pd.concat([pd.read_csv('s3://' + bucket + '/' + obj) for obj in objs[1:]])

X = train_df.copy()
X.drop(['sk_id_curr', 'target'], axis=1, inplace=True)
y = train_df['target'].copy()
X.fillna(0, inplace=True)

CPU times: user 16.8 s, sys: 5.47 s, total: 22.3 s
Wall time: 55 s


In [5]:
print((train_df.shape, X.shape, y.shape))

((306020, 376), (306020, 374), (306020,))


In [6]:
feature_dir_local = 'trnval'

if not os.path.exists(feature_dir_local):
    os.mkdir(feature_dir_local)

<br>

## 2. K-fold Split and Save
----
- 데이터를 학습 데이터와 검증 데이터로 분리하여 libsvm 포맷으로 로컬에 저장 후, S3로 업로드합니다.
    - Stratified K-Fold(층화추출)로 fold를 생성합니다. 이 방법은 특정 fold에 특정 class가 몰리는 현상을 방지할 수 있기에 불균형 데이터(imbalanced dataset)의 검증에 적절합니다.
    - Sagemaker의 XGBoost에서 지원하는 입력 데이터 포맷은 *text/libsvm*(Default) 또는 *text/csv* 입니다.
    - libsvm포맷의 각 열은 `<label> <index0>:<value0> <index1>:<value1> ...` 형식으로 저장되며 binary 포맷은 아니지만, csv 파일 대비 용량을 절약할 수 있습니다.
- 본 데모에선 편의상 하나의 fold만 저장합니다. 만약 모든 fold를 저장하고 싶다면 `if fold_ == 0:`을 주석처리하시면 됩니다.

In [7]:
from sklearn.model_selection import StratifiedKFold, train_test_split, ShuffleSplit
from sklearn.datasets import dump_svmlight_file
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### Save features for XGBoost, a built-in algorithm in Sagemaker

In [9]:
%%time
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X.values, y.values)):
    if fold_ == 0:    
        # split training data and validation data
        trn_data, trn_y = X.iloc[trn_idx], y.iloc[trn_idx]
        val_data, val_y = X.iloc[val_idx], y.iloc[val_idx]

        # save features to local     
        print('---> Dumping fold_{} to local'.format(fold_))
        trn_path_local = os.path.join(feature_dir_local, 'train.libsvm')
        val_path_local = os.path.join(feature_dir_local, 'valid.libsvm')  
        print('\t XGBoost train_path={} \n\t XGBoost valid_path={}'.format(trn_path_local, val_path_local))

        dump_svmlight_file(X=trn_data, y=trn_y, f=trn_path_local)
        dump_svmlight_file(X=val_data, y=val_y, f=val_path_local)

        # upload to amazon s3
        print('---> Saving fold_{} to s3 bucket'.format(fold_))
        trn_path_s3 = os.path.join(prefix_target, 'train.libsvm')
        val_path_s3 = os.path.join(prefix_target, 'valid.libsvm')
        print('\t S3 train_path={} \n\t S3 valid_path={}'.format(trn_path_s3, val_path_s3))    

        boto3.Session().resource('s3').Bucket(bucket).Object(trn_path_s3).upload_file(trn_path_local)
        boto3.Session().resource('s3').Bucket(bucket).Object(val_path_s3).upload_file(val_path_local)
        
        # for checking prediction results using local model, instead of sagemaker container
        joblib.dump((trn_data, val_y), os.path.join(feature_dir_local, 'train.pkl'))
        joblib.dump((val_data, trn_y), os.path.join(feature_dir_local, 'valid.pkl'))             

---> Dumping fold_0 to local
	 XGBoost train_path=trnval/train.libsvm 
	 XGBoost valid_path=trnval/valid.libsvm
---> Saving fold_0 to s3 bucket
	 S3 train_path=homecredit/modeling-feature-190808/train.libsvm 
	 S3 valid_path=homecredit/modeling-feature-190808/valid.libsvm
CPU times: user 2min 8s, sys: 5.03 s, total: 2min 13s
Wall time: 2min 14s


### Save features for LightGBM
- LightGBM은 Binary 포맷으로 변환 가능하여 용량을 대폭 줄일 수 있습니다. 아래 python code snippet을 참조해 주세요.
```
trn_lgb = lgb.Dataset(trn_data, label=trn_y)
trn_lgb.save_binary(YOUR_PATH)
```

In [10]:
%%time
import lightgbm as lgb
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X.values, y.values)):
    if fold_ == 0:    
        # split training data and validation data
        trn_data, trn_y = X.iloc[trn_idx], y.iloc[trn_idx]
        val_data, val_y = X.iloc[val_idx], y.iloc[val_idx]

        # save features to local     
        print('---> Dumping fold_{} to local'.format(fold_))
        trn_path_local = os.path.join(feature_dir_local, 'train.bin')
        val_path_local = os.path.join(feature_dir_local, 'valid.bin')  
        print('\t LightGBM train_path={} \n\t LightGBM valid_path={}'.format(trn_path_local, val_path_local))

        trn_lgb = lgb.Dataset(trn_data, label=trn_y)
        val_lgb = lgb.Dataset(val_data, label=val_y)
        
        trn_lgb.save_binary(trn_path_local)
        val_lgb.save_binary(val_path_local)

        # upload to amazon s3
        print('---> Saving fold_{} to s3 bucket'.format(fold_))
        trn_path_s3 = os.path.join(prefix_target, 'train.bin')
        val_path_s3 = os.path.join(prefix_target, 'valid.bin')
        print('\t S3 train_path={} \n\t S3 valid_path={}'.format(trn_path_s3, val_path_s3))    

        boto3.Session().resource('s3').Bucket(bucket).Object(trn_path_s3).upload_file(trn_path_local)
        boto3.Session().resource('s3').Bucket(bucket).Object(val_path_s3).upload_file(val_path_local)  

---> Dumping fold_0 to local
	 LightGBM train_path=trnval/train.bin 
	 LightGBM valid_path=trnval/valid.bin
---> Saving fold_0 to s3 bucket
	 S3 train_path=homecredit/modeling-feature-190808/train.bin 
	 S3 valid_path=homecredit/modeling-feature-190808/valid.bin
CPU times: user 17.3 s, sys: 2.83 s, total: 20.1 s
Wall time: 8.21 s
